In [1]:
from zenml import step
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

@step
def local_data_loading() -> dict:
    """Load the iris dataset from scikit-learn."""
    iris = load_iris()
    X = iris.data
    y = iris.target
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}


In [2]:
from zenml import step
from sklearn.linear_model import LogisticRegression

@step(step_operator="k8s")
def remote_model_training(data: dict) -> LogisticRegression:
    """Train a logistic regression model on the iris dataset."""
    model = LogisticRegression()
    model.fit(data['X_train'], data['y_train'])
    
    train_accuracy = model.score(data['X_train'], data['y_train'])
    test_accuracy = model.score(data['X_test'], data['y_test'])
    
    print(f"Train accuracy: {train_accuracy:.2f}")
    print(f"Test accuracy: {test_accuracy:.2f}")
    
    return model

In [3]:
from zenml import pipeline
from zenml.config import DockerSettings

@pipeline(settings={"docker": DockerSettings(required_integrations=["sklearn"], parent_image="zenml:parent-image")})
def local_and_remote_pipeline():
    """Define a pipeline that connects the steps."""
    data = local_data_loading()
    model = remote_model_training(data)

In [4]:
run = local_and_remote_pipeline()
print("success")

Initiating a new run for the pipeline: local_and_remote_pipeline.
Extracting step code from nodebook cell into file /home/htahir1/workspace/zenml_io/zenml-tests/__remote_model_training.py.
Registered new version: (version 1).
The stack aws-kubernetes contains components that require building Docker images. Older versions of ZenML always built these images locally, but since version 0.32.0 this behavior can be configured using the image_builder stack component. This stack will temporarily default to a local image builder that mirrors the previous behavior, but this will be removed in future versions of ZenML. Please add an image builder to this stack:
zenml image-builder register <NAME> ...
zenml stack update ffb7f641-fd0e-4f31-ad03-40dad8362450 -i <NAME>
Unable to use code repository to download code for this run as there are uncommitted changes.
Unable to find a build to reuse. When using a code repository, a previous build can be reused when the following conditions are met:
  * The 

RuntimeError: Pod `zenml:local-and-remote-pipeline-2024-07-25-12-11-36-067664-remote-model-training` failed.

In [ ]:
print(run.steps["remote_model_training"].output.load())